# <span style="color:purple">Basic **WRDS** Queries</span>

Import the `wrds` package in python, along with any other packages you will use.

In [1]:
# packages
import wrds
import pandas as pd

Once you have a hidden pgpass file saved, you will not have to enter your crednetials the next time you login to the **WRDS** API in python.

In [2]:
conn = wrds.Connection(wrds_username='ambreen')

Loading library list...
Done


#### Basic Query

In a few instances, you can extract a full data table. 

In [3]:
company = conn.get_table(library='comp', table='company')
print(company.shape)

(53413, 39)


In [4]:
company = pd.DataFrame(company)
company[0:5]

,conm,gvkey,add1,add2,add3,add4,addzip,busdesc,cik,city,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,A & E PLASTIK PAK INC,001000,None,None,None,None,None,A & E Plastik Pak Inc. is a commodity chemical...,None,None,...,01,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
1,A & M FOOD SERVICES INC,001001,1924 South Utica Avenue,None,None,None,94104,None,0000723576,Tulsa,...,01,5812,420.0,978.0,None,OK,0.0,None,1986-07-31,None
2,AAI CORP,001002,124 Industry Lane,None,None,None,21030-0126,"Textron Systems Corporation designs, develops,...",0001306124,Hunt Valley,...,01,3825,230.0,940.0,None,MD,0.0,www.textronsystems.com,1977-03-31,None
3,A.A. IMPORTING CO INC,001003,7700 Hall Street,None,None,None,63125,"A.A. Importing Company, Inc. designs, manufact...",0000730052,St. Louis,...,01,5712,449.0,976.0,None,MO,3.0,www.aaimporting.com,1992-04-30,None
4,AAR CORP,001004,"One AAR Place, 1100 North Wood Dale Road",None,None,None,60191,AAR Corp. provides products and services to co...,0000001750,Wood Dale,...,01,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,None,1972-04-24


You can also subset a table to only certain columns.

In [5]:
company2 = conn.get_table(library='comp', table='company', columns = ['conm', 'gvkey', 'cik', 'add1', 'add2', 'naics'])
company2 = pd.DataFrame(company2)
company2[0:5]

,conm,gvkey,cik,add1,add2,naics
0,A & E PLASTIK PAK INC,001000,None,None,None,None
1,A & M FOOD SERVICES INC,001001,0000723576,1924 South Utica Avenue,None,722
2,AAI CORP,001002,0001306124,124 Industry Lane,None,None
3,A.A. IMPORTING CO INC,001003,0000730052,7700 Hall Street,None,442110
4,AAR CORP,001004,0000001750,"One AAR Place, 1100 North Wood Dale Road",None,423860


#### Combine a Complete table with another dataset

If you are able to extract a full WRDS table you can combine the data with an existing dataset you have.  In the example below, we have `gvkeys` for both Apple and Microsoft, respectively.

In [6]:
# selecting rows based on condition
key_options = ['001690', '012141']

company2 = company[company['gvkey'].isin(key_options)]
print(company2)

                 conm   gvkey                add1  add2  add3  add4  \
684         APPLE INC  001690  One Apple Park Way  None  None  None   
10933  MICROSOFT CORP  012141   One Microsoft Way  None  None  None   

           addzip                                            busdesc  \
684         95014  Apple Inc. designs, manufactures, and markets ...   
10933  98052-6399  Microsoft Corporation develops and supports so...   

              cik       city  ... priusa   sic spcindcd spcseccd spcsrc state  \
684    0000320193  Cupertino  ...     01  3663    190.0    940.0     A+    CA   
10933  0000789019    Redmond  ...     01  7372    185.0    940.0     A+    WA   

      stko             weburl dldte     ipodate  
684    0.0      www.apple.com  None  1980-12-12  
10933  0.0  www.microsoft.com  None  1986-03-13  

[2 rows x 39 columns]


#### Iterating over a full table

For most tables, you can extract the full contents by iterating over observations using the `obs` and `offset` options.

In [7]:
# Extract the key financials table in increments for 1,000 companies 
# at a time.
orbis1 = conn.get_table(library='bvd_orbis_large', table='ob_key_financials_l', obs=1000)
orbis2 = conn.get_table(library='bvd_orbis_large', table='ob_key_financials_l', obs=1000, offset=1001)

In [8]:
print(orbis1.shape)
print(orbis2.shape)

# convert to pandas dataframe
orbis1 = pd.DataFrame(orbis1)
orbis2 = pd.DataFrame(orbis2)

# save the results
orbis1.to_csv('~/Desktop/orbis1.csv', encoding='utf-8')
orbis2.to_csv('~/Desktop/orbis2.csv', encoding='utf-8')

(1000, 28)
(1000, 28)


#### Using SQL Queries to Merge Datasets

Finally, you can merge datasets using SQL queries on the WRDS tables.

In [9]:
apple_fund = conn.raw_sql("""select a.gvkey, a.iid, a.datadate, a.tic, a.conm,
                            a.at, b.prccm, b.cshoq 
                            
                            from comp.funda a 
                            inner join comp.secm b 
                            
                            on a.gvkey = b.gvkey
                            and a.iid = b.iid
                            and a.datadate = b.datadate
                        
                            where a.tic = 'AAPL' 
                            and a.datadate>='01/01/2010'
                            and a.datafmt = 'STD' 
                            and a.consol = 'C' 
                            and a.indfmt = 'INDL'
                            """, date_cols=['datadate'])

apple_fund.shape

(14, 8)

In [10]:
apple_fund 

,gvkey,iid,datadate,tic,conm,at,prccm,cshoq
0,001690,01,2010-09-30,AAPL,APPLE INC,75183.0,283.750,915.970
1,001690,01,2011-09-30,AAPL,APPLE INC,116371.0,381.320,929.277
2,001690,01,2012-09-30,AAPL,APPLE INC,176064.0,667.105,939.208
3,001690,01,2013-09-30,AAPL,APPLE INC,207000.0,476.750,899.213
4,001690,01,2014-09-30,AAPL,APPLE INC,231839.0,100.750,5866.161
5,001690,01,2015-09-30,AAPL,APPLE INC,290479.0,110.300,5578.753
6,001690,01,2016-09-30,AAPL,APPLE INC,321686.0,113.050,5336.166
7,001690,01,2017-09-30,AAPL,APPLE INC,375319.0,154.120,5126.201
8,001690,01,2018-09-30,AAPL,APPLE INC,365725.0,225.740,4754.986
9,001690,01,2019-09-30,AAPL,APPLE INC,338516.0,223.970,4443.236


In [11]:
apple2 = conn.raw_sql('''select conm, gvkey, cik FROM comp.funda WHERE fyear>2010 AND (tic='AAPL')''')
apple2

,conm,gvkey,cik
0,APPLE INC,001690,0000320193
1,APPLE INC,001690,0000320193
2,APPLE INC,001690,0000320193
3,APPLE INC,001690,0000320193
4,APPLE INC,001690,0000320193
5,APPLE INC,001690,0000320193
6,APPLE INC,001690,0000320193
7,APPLE INC,001690,0000320193
8,APPLE INC,001690,0000320193
9,APPLE INC,001690,0000320193


In [12]:
AAPL_bvid = 'US942404110' 
apple3 = conn.raw_sql('''select * FROM bvd_orbis_large.ob_key_financials_l WHERE closdate_year>2010 AND (bvdid='US942404110')''')
apple3

,bvdid,category_of_company,ctryiso,conscode,filing_type,closdate,closdate_year,nr_months,audstatus,accpractice,...,toas,shfd,curr,prma,rshf,rcem,solr,_611,empl,astk_market_cap
0,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2011-09-24,2011.0,12,Unqualified,US GAAP,...,1.163710e+11,7.661500e+10,1.61,31.60,44.65,NaN,65.84,14.46,60400.0,374823.23
1,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2012-09-29,2012.0,12,Unqualified,US GAAP,...,1.760640e+11,1.182100e+11,1.50,35.63,47.17,NaN,67.14,14.99,72800.0,625348.84
2,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2013-09-28,2013.0,12,Unqualified,US GAAP,...,2.070000e+11,1.235490e+11,1.68,29.35,40.60,30.79,59.69,11.69,80300.0,433126.41
3,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2014-09-27,2014.0,12,Unqualified,US GAAP,...,2.318390e+11,1.115470e+11,1.08,29.26,47.95,31.99,48.11,15.27,92600.0,603277.60
4,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2015-09-26,2015.0,12,Unqualified,US GAAP,...,2.903450e+11,1.193550e+11,1.11,31.03,60.76,34.92,41.11,11.78,110000.0,629010.25
5,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2016-09-24,2016.0,12,Unqualified,US GAAP,...,3.216860e+11,1.282490e+11,1.35,28.46,47.85,25.89,39.87,13.29,116000.0,607331.41
6,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2017-09-30,2017.0,12,Unqualified,US GAAP,...,3.753190e+11,1.340470e+11,1.28,27.96,47.81,24.19,35.72,16.46,123000.0,796064.91
7,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2018-09-29,2018.0,12,Unqualified,US GAAP,...,3.657250e+11,1.071470e+11,1.13,27.45,68.04,30.48,29.30,18.32,132000.0,1090307.52
8,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2019-09-28,2019.0,12,Unqualified,US GAAP,...,3.385160e+11,9.048800e+10,1.54,25.27,72.65,29.77,26.73,18.32,137000.0,1012160.75
9,US942404110,VERY LARGE COMPANY,US,C1,Annual report,2020-09-26,2020.0,12,Unqualified,US GAAP,...,3.238880e+11,6.533900e+10,1.36,24.44,102.68,32.02,20.17,34.97,147000.0,2007837.30


In [13]:
AAPL_bvid = 'US942404110' 
MSFT_bvid = 'JO30191GJ'

orbis_merged = conn.raw_sql('''select * FROM bvd_orbis_large.ob_key_financials_l WHERE closdate_year>2010 AND (bvdid in ('US942404110', 'JO30191GJ'))''')
orbis_merged

,bvdid,category_of_company,ctryiso,conscode,filing_type,closdate,closdate_year,nr_months,audstatus,accpractice,...,toas,shfd,curr,prma,rshf,rcem,solr,_611,empl,astk_market_cap
0,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2011-12-31,2011.0,12,Unqualified,IFRS,...,2.450000e+07,1.777400e+07,0.66,-2.31,-1.32,1.17,72.55,NaN,250.0,4.46
1,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2012-12-31,2012.0,12,Unqualified,IFRS,...,2.789500e+07,1.891700e+07,0.73,7.33,6.68,8.47,67.82,10.48,284.0,11.96
2,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2013-12-31,2013.0,12,Unqualified,IFRS,...,2.857300e+07,2.004000e+07,0.79,6.64,6.70,7.88,70.14,12.41,220.0,13.92
3,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2014-12-31,2014.0,12,Unqualified,IFRS,...,2.980300e+07,2.058100e+07,0.82,8.78,8.35,9.80,69.06,8.96,337.0,12.85
4,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2015-12-31,2015.0,12,Unqualified,IFRS,...,3.035500e+07,2.101600e+07,0.76,6.98,7.13,9.04,69.23,8.83,341.0,10.18
5,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2016-12-31,2016.0,12,Unqualified,IFRS,...,3.211600e+07,2.189600e+07,0.73,5.44,5.49,7.22,68.18,10.87,369.0,10.36
6,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2017-12-31,2017.0,12,Unqualified,IFRS,...,3.464700e+07,2.202300e+07,0.60,4.49,5.22,7.67,63.56,9.91,362.0,8.69
7,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2018-12-31,2018.0,12,Unqualified,IFRS,...,3.536300e+07,2.161300e+07,0.74,3.43,4.25,6.90,61.12,8.86,383.0,5.92
8,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2019-12-31,2019.0,12,Unqualified,IFRS,...,3.376000e+07,2.181000e+07,0.85,3.75,4.62,7.81,64.60,8.19,NaN,6.29
9,JO30191GJ,VERY LARGE COMPANY,JO,C1,Annual report,2020-12-31,2020.0,12,Unqualified,IFRS,...,3.220500e+07,2.198700e+07,1.07,1.44,1.26,3.87,68.27,32.35,331.0,5.92
